let us start by importing the required functions

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
# import statsemodels.api as sm
import SALib
import seaborn as sns
import mpl_toolkits.axisartist as AA
import mpld3
import scipy as sp
import copy
import matplotlib.ticker as ticker

from ema_workbench import (Model, RealParameter, IntegerParameter, Policy, CategoricalParameter, Constant, TimeSeriesOutcome, ScalarOutcome, perform_experiments, ema_logging, save_results, load_results)
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator
from ema_workbench.connectors.pysd_connector import PysdModel 
from ema_workbench.connectors import vensimDLLwrapper
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.util import load_results
from ema_workbench.util import ema_logging

from ema_workbench.analysis import prim
from ema_workbench.analysis.plotting import lines, multiple_densities, kde_over_time
from ema_workbench.analysis.plotting_util import KDE
from ema_workbench.analysis.plotting_util import determine_time_dimension
from ema_workbench.analysis.pairs_plotting import pairs_scatter

from SALib.sample import saltelli
from SALib.analyze import sobol, morris
from SALib.test_functions import Ishigami

from mpl_toolkits.axes_grid1 import host_subplot

from scipy.stats import linregress

ema_logging.log_to_stderr(ema_logging.INFO)

C:\Users\guri_\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:18: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)


<Logger EMA (DEBUG)>

# Adding Reconversion Open Exploration  

In [2]:
vensimModel = VensimModel("Step4Modelshort", model_file="ShortTermPolicyforObjectives_Step4_shortwithPolicies.vpm" )

In [3]:
vensimModel.uncertainties = [

            RealParameter("Average Fertility Rate", 0.01, 0.025),
            RealParameter("Average Mortality Rate", 0.00126, 0.00154),
            IntegerParameter("BASE Average Household size", 2, 7),     
            RealParameter("average Immigration percent", 0.01, 0.06),
            RealParameter("Housing Area Per person", 0.000004, 0.000006),
            RealParameter("Floor Area Ratio", 2, 7),
            RealParameter("Average Residential approval period", 0.5, 2),
            RealParameter("Base Green Area commisioned per year", 0.02, 0.06),
            RealParameter("average green area construction period", 0.5, 2),
            RealParameter("Base Economic area Conversion rate", 0.01, 0.06),
            RealParameter("Base Slum reconversion rate", 0.01, 0.06),
            RealParameter("Fraction of people selling houses when leaving city", 0.1, 0.5),
            RealParameter("house construction time", 0.5, 2),
            IntegerParameter("time delay effect uncertainty of spending capacity on housing price", 1, 4),
            RealParameter("Slum dwelling area per person", 0.000004, 0.000006),
            RealParameter("Base residential conversion rate", 0.01, 0.05),
            RealParameter("average Emmigration percent", 0.008, 0.05),
            RealParameter("Relocation Rate", 0.00, 0.02), #for relocation of population on streets
            RealParameter("Average Commercial Approval Period", 0.5, 2),
            RealParameter("Base Green Area Reconversion rate", 0.001, 0.03),
            RealParameter("Average Slum Construction period", 0.1, 0.5),
            RealParameter("Average slum Demolition Period", 0.5, 2),
            RealParameter("uncertainty of average percent of inflation effect", 0.01, 0.08),
            RealParameter("average payment to income ratio for housing", 0.2, 0.5),
            RealParameter("Base Non Economic Reconv rate", 0.01 ,0.05),
            RealParameter("Ratio of Renting to Buying",0.2, 1),
            RealParameter("margin of empty housing",0.1,0.5)
]

vensimModel.levers= [
            RealParameter("Policy High rise Program switch",0,1),
            RealParameter("increase in FAR",1,5),
            RealParameter("FAR implementation year",2020,2030),
    
            RealParameter("Accelerated city growth program switch",0,1),
            RealParameter("effect of green recov",1,5),
            RealParameter("Green reconv policy start",2020,2030),
    
            RealParameter("Policy reconversion of Slums Switch",0,1),
            RealParameter("effect of reconversion rate on Policy",1,5),
            RealParameter("Slum reconv polict start year",2020,2030),
    
            RealParameter("Austerity Policy Program Switch",0,1),
            RealParameter("effect on non resi reconv of policy",1,5),
            RealParameter("reconv policy start year",2020,2030),
            RealParameter("Austerity effect on inflation",1,5),
            RealParameter("Auterity implementation year",2020,2030),
    
            RealParameter("Housing Finance Policy Program Switch",0,1),
            RealParameter("Housing finance effect on household spendings",1,5),
            RealParameter("Housing Finane Policy start year",2020,2030),
    
            RealParameter("Policy Relocation program Switch",0,1),
            RealParameter("effect on relocation rate",1,5),
            RealParameter("additional relocation housing factor",1,5),
            RealParameter("Relocation policy start year",2020,2030),
]

vensimModel.outcomes = [
    TimeSeriesOutcome('Housing Affordability Rating'),
    TimeSeriesOutcome('People with No Settlements'),
    TimeSeriesOutcome('Illegal Slum Population'),
    TimeSeriesOutcome('Urban Non Residential Area'),
    TimeSeriesOutcome('Green Area')
    
]
    
MAXIMIZE = ScalarOutcome.MAXIMIZE   
MINIMIZE = ScalarOutcome.MINIMIZE 

vensimModel.robustness_minimax_maximin = [ScalarOutcome('Housing Affordability Rating' , kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('People with No Settlements', kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('Illegal Slum Population', kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('Urban Non Residential Area', kind=MAXIMIZE, function=np.min),
                                          ScalarOutcome('Green Area', kind=MAXIMIZE, function=np.min)
                                          ]

In [4]:
n_scenarios = 50
scenarios = sample_uncertainties(vensimModel, n_scenarios)
nfe = 1000
    
with SequentialEvaluator(vensimModel) as evaluator:
    b = evaluator.robust_optimize(vensimModel.robustness_minimax_maximin, scenarios, nfe=nfe, population_size=25,
                                  epsilons=[0.1,]*len(vensimModel.robustness_minimax_maximin) )

[MainProcess/INFO] performing 50 scenarios * 25 policies * 1 model(s) = 1250 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 417 cases completed
[MainProcess/INFO] 834 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 433 cases completed
[MainProcess/INFO] 866 cases completed
[MainProcess/INFO] 1299 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 433 cases completed
[MainProcess/INFO] 866 cases completed
[MainProcess/INFO] 1299 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments seq

In [5]:
b.to_csv('./Results_21_solutions_minimax_maximin_shortpolicies4.csv')

In [3]:
import pandas as pd
df_minimax_maximin = pd.DataFrame.from_csv('./Results_21_solutions_minimax_maximin_shortpolicies4.csv')

C:\Users\guri_\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  from ipykernel import kernelapp as app


In [4]:
df_minimax_maximin.head()

,Policy High rise Program switch,increase in FAR,FAR implementation year,Accelerated city growth program switch,effect of green recov,Green reconv policy start,Policy reconversion of Slums Switch,effect of reconversion rate on Policy,Slum reconv polict start year,Austerity Policy Program Switch,...,Housing finance effect on household spendings,Housing Finane Policy start year,Policy Relocation program Switch,effect on relocation rate,additional relocation housing factor,Housing Affordability Rating,People with No Settlements,Illegal Slum Population,Urban Non Residential Area,Green Area
0,0.244798,1.827934,2028.512104,0.898308,4.462110,2024.052294,0.993820,2.703666,2024.215898,0.496217,...,4.419078,2020.779895,0.917147,4.982145,4.039512,10.175423,2825471.75,600000.0,69.366280,4.038444
1,0.949481,4.191758,2028.594542,0.896433,4.427545,2029.129188,0.934723,4.165695,2021.840554,0.142355,...,4.168802,2020.094656,0.909216,4.973142,3.206129,9.922953,2894172.50,600000.0,75.609741,3.911556
2,0.825753,4.171885,2028.546936,0.558835,4.058515,2029.193019,0.936673,4.166523,2021.027757,0.134943,...,4.179686,2020.273679,0.903193,4.161032,2.101807,9.993089,3100134.25,600000.0,77.331528,3.834682
3,0.914397,4.346740,2028.491910,0.925600,4.530834,2026.644786,0.995609,4.430664,2021.857041,0.202471,...,4.415594,2021.639353,0.888587,4.974941,3.108352,10.512746,3607957.50,600000.0,80.685913,3.134280
4,0.877169,4.137027,2029.703816,0.895533,4.546891,2026.635887,0.995351,4.388921,2020.332162,0.136214,...,4.348940,2020.375557,0.915829,4.892751,3.108803,10.022420,2932358.00,600000.0,77.005363,3.908856


In [5]:
import plotly

In [6]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In [7]:
# create the parcoord

df_parallel = df_minimax_maximin

data = [
    Parcoords(
        line = dict(color = df_parallel['Housing Affordability Rating'],
                   colorscale = 'Jet',
                   showscale = True,
                   reversescale = False,
                   cmin = df_parallel['Housing Affordability Rating'].min(),
                   cmax = df_parallel['Housing Affordability Rating'].max()),

        dimensions = list([
             dict(label ='People with No Settlements'  ,
                values =df_parallel['People with No Settlements']  ),
            dict(label ='Illegal Slum Population'  ,
                values =df_parallel['Illegal Slum Population']  ),
            dict(label = 'Urban Non Residential Area',
                values = df_parallel['Urban Non Residential Area']),
            dict(label = 'Housing Affordability Rating',
                values = df_parallel['Housing Affordability Rating']),
            dict(label = 'Green Area',
                values = df_parallel['Green Area'])
            
        ])
    )
]


layout = Layout(
    title = "Parallel coordinates plot"

)
fig = Figure(data=data, layout=layout)
iplot(fig)